In [1]:
from PIL import Image, ImageFont, ImageDraw
import matplotlib.pyplot as plt
import pyrealsense2 as rs
from scipy import ndimage
import numpy as np
import glob as gb
import pyyolo
import math
import time
import os
import darknet
import cv2

In [2]:
pipeline = rs.pipeline()  #接收相机信息
config = rs.config()      #相机信息配置
config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)   #848*480
config.enable_stream(rs.stream.color, 848, 480, rs.format.bgr8, 30)

profile = pipeline.start(config)   #打开相机
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()   #检索深度图的单位和meters单位之间的映射 (深度比例)

align_to = rs.stream.color
align = rs.align(align_to)  #执行深度图像与另一个图像之间的对其
def get_aligned_imgs():
    for i in range(50):
        frames = pipeline.wait_for_frames()     #获取Realsense一帧的数据
        aligned_frames = align.process(frames)   #在给定的帧上运行对齐过程，以获得一组对齐的帧

        depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        cv2.imwrite('/home/toke/Desktop/image/camera_catch.jpg', color_image)
        cv2.imwrite('/home/toke/Desktop/image/camera_catch_depth.jpg', depth_image)
    return color_image, depth_image
color_image, depth_image = get_aligned_imgs()

In [3]:
depth_image

array([[4359, 4332, 4332, ...,  476,  475,  473],
       [4387, 4387, 4332, ...,  476,  475,  473],
       [4416, 4416, 4387, ...,  477,  476,  474],
       ...,
       [ 301,  301,  301, ...,  363,  363,  363],
       [ 301,  301,  301, ...,  362,  362,  362],
       [ 300,  300,  300, ...,  362,  362,  362]], dtype=uint16)

In [4]:
testImg = '/home/toke/Desktop/image/camera_catch.jpg'
### python3 run yolov4 model
img = Image.open(testImg)   #PIL库的图片读取函数 （W x H 不含通道数）
fname = os.path.basename(testImg)
draw = ImageDraw.Draw(img)  #在图像上绘制一些东西
image = cv2.imread(testImg) 

detector = pyyolo.YOLO("/home/toke/Packages/darknet/models/yolov4-custom.cfg",
                       "/home/toke/Packages/darknet/models/yolov4-custom_45000_2020_0629.weights",
                       "/home/toke/Packages/darknet/models/my_laji.data",
                       detection_threshold = 0.5,
                       hier_threshold = 0.5,
                       nms_threshold = 0.45)   
#nms_threshold(非极大值抑制)
print(detector)
bboxes, center_points = [], []
dets = detector.detect(image, rgb=False)
print(dets)
for i, det in enumerate(dets):
    # print ('Detection:', {i}, {det})
    xmin, ymin, xmax, ymax = det.to_xyxy()
    bbox = [xmin, ymin, xmax, ymax]

    center_point = (math.ceil((xmin + xmax) / 2), math.ceil((ymin + ymax) / 2))
    draw.ellipse((center_point[0], center_point[1], center_point[0] + 20, center_point[1] + 20), 'red')

bboxes = np.vstack(bbox)
center_points = np.vstack(center_point)

img.show()
path = '/home/toke/Desktop'
fname = os.path.basename(testImg)
img.save(os.path.join(path, fname))

#return bboxes, center_points

[BBox([351, 136,  60, 131])]


In [5]:
bbox_pick, center_points = bboxes, center_point    

In [6]:
bbox_pick

array([[351],
       [136],
       [411],
       [267]])

In [7]:
depth_crop = depth_image[int(bbox_pick[1]):int(bbox_pick[3]), int(bbox_pick[0]):int(bbox_pick[2])]

In [8]:
depth_image

array([[4359, 4332, 4332, ...,  476,  475,  473],
       [4387, 4387, 4332, ...,  476,  475,  473],
       [4416, 4416, 4387, ...,  477,  476,  474],
       ...,
       [ 301,  301,  301, ...,  363,  363,  363],
       [ 301,  301,  301, ...,  362,  362,  362],
       [ 300,  300,  300, ...,  362,  362,  362]], dtype=uint16)

In [9]:
crop_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_crop, alpha=0.03), cv2.COLORMAP_JET)
cv2.imshow("cropped depth", crop_colormap)

In [10]:
depth_crop

array([[   0,    0,    0, ..., 2950, 2950, 2963],
       [   0,    0,    0, ..., 2950, 2950, 2950],
       [   0,    0,    0, ..., 2925, 2925, 2925],
       ...,
       [ 837,  837,  834, ...,  858,  858,  865],
       [ 835,  835,  834, ...,  858,  858,  865],
       [ 835,  835,  833, ...,  858,  858,  866]], dtype=uint16)

In [11]:
# y=depth_crop.tolist()
y = depth_crop.reshape(-1,1)

In [12]:
np.shape(y)

(7860, 1)

In [13]:
import sklearn.metrics as metrics
# 构造自定义函数，用于绘制不同k值和对应轮廓系数的折线图
def k_silhouette(X, clusters):
    K = range(2,clusters+1)
# 构建空列表，用于存储个中簇数下的轮廓系数
    S = []
    for k in K:
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(X)
        labels = kmeans.labels_
        S.append(metrics.silhouette_score(X, labels, metric='euclidean'))  # 调用字模块metrics中的silhouette_score函数，计算轮廓系数
   
    plt.rcParams['font.sans-serif'] = ['KaiTi', 'SimHei', 'FangSong']# 中文和负号的正常显示
    plt.rcParams['axes.unicode_minus'] = False

    plt.style.use('ggplot')# 设置绘图风格
    
    plt.plot(K, S, 'b*-')  # 绘制K的个数与轮廓系数的关系
    plt.xlabel('the number of chu')
    plt.ylabel('lun kuo xi shu')
    
    plt.show() # 显示图形


k_silhouette(y, 50)# 自定义函数的调用（指定原始数据和选取范围）

NameError: name 'KMeans' is not defined

In [ ]:
from sklearn.cluster import KMeans
for k in range(2,5):
    clf = KMeans(n_clusters=k) #设定k  ！！！！！！！！！！这里就是调用KMeans算法
    s = clf.fit(y) #加载数据集合
    numSamples = len(y) 
    centroids = clf.labels_
    print (centroids,type(centroids)) #显示中心点
    print (clf.inertia_)  #显示聚类效果 评价指标
    mark = ['or', 'ob', 'og', 'ok', '^r', '+r', 'sr', 'dr', '<r', 'pr']
    #画出所有样例点 属于同一分类的绘制同样的颜色
    for i in range(numSamples):
        #markIndex = int(clusterAssment[i, 0])
        plt.plot(y[i][0], 0, mark[clf.labels_[i]]) #mark[markIndex])
    mark = ['Dr', 'Db', 'Dg', 'Dk', '^b', '+b', 'sb', 'db', '<b', 'pb']
    # 画出质点，用特殊图型
    centroids =  clf.cluster_centers_
    for i in range(k):
        plt.plot(centroids[i][0], 0, mark[i], markersize = 12)
        #print centroids[i, 0], centroids[i, 1]
        plt.axvline(centroids[i][0], ymin=0.3, ymax=0.7)
    plt.show()

In [ ]:
plt.imshow(depth_crop)

In [ ]:
np.shape(depth_image)

In [ ]:
depth_image[460][447]
Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.

In [ ]:
depth_image[int(bbox_pick[0])][int(bbox_pick[1])]

In [ ]:
bbox_pick

In [ ]:
bbox_pick[1]

In [ ]:
def calculateCoM(dpt):   #计算质心
    """
    Calculate the center of mass
    :param dpt: depth image
    :return: (x,y,z) center of mass
    """

    dc = dpt.copy()
    dc[dc < 0] = 0
    dc[dc > 10000] = 0
    cc = ndimage.measurements.center_of_mass(dc > 0)
    print ('cc:', cc)
    num = np.count_nonzero(dc)
    com = np.array((cc[1]*num, cc[0]*num, dc.sum()), np.float)

    if num == 0:
        return np.array((0, 0, 0), np.float)
    else:
        return com/num

In [ ]:
mass_center = calculateCoM(depth_crop) 

In [ ]:
mass_center

In [ ]:
y = depth_crop[int((mass_center[0]-5)):int((mass_center[0]+5)), int((mass_center[1]-5)):int((mass_center[1]+5))]
y = y.reshape(-1,1)

print(y)

In [ ]:
import numpy as np
x = np.zeros(5) 

In [ ]:
x

In [ ]:
x[1] = 10

In [ ]:
x[2] = 11

In [ ]:
x

In [14]:
dets

[BBox([351, 136,  60, 131])]

In [18]:
dets[0][2]

60

In [ ]:
setting an array element with a sequence.